# Memory Poisoning Resistance Analysis

## 1. Introduction

"""
Ce notebook documente les tests de résistance au Memory Poisoning de MemStream.

### Objectifs:
1. Tester si le seuil β empêche l'ajout d'anomalies à la mémoire
2. Évaluer l'impact de différents paramètres (β, γ) sur la protection
3. Identifier les scénarios d'attaque les plus dangereux

### Méthodologie:
- Initialiser une mémoire avec des encodings normaux
- Simuler une attaque avec des encodings anormaux
- Mesurer le taux de protection et de contamination
"""

## 2. Setup


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.memory import MemoryModule

In [ ]:
# Configuration de base
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 3. Génération des Données

### 3.1 Données Normales (Mémoire Initiale)

In [ ]:
memory_size = 50
encoding_dim = 5

# Générer des encodings normaux (distribution N(0,1))
normal_encodings = np.random.randn(memory_size, encoding_dim)

print(f"Encodings normaux générés:")
print(f"  Shape: {normal_encodings.shape}")
print(f"  Mean: {normal_encodings.mean(axis=0)}")
print(f"  Std: {normal_encodings.std(axis=0)}")

In [ ]:
# Visualiser la distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Distribution de la première dimension
axes[0].hist(normal_encodings[:, 0], bins=20, alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Valeur')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution - Dimension 0')
axes[0].axvline(x=0, color='r', linestyle='--', label='Moyenne')
axes[0].legend()

# Scatter plot 2D (dimensions 0 et 1)
axes[1].scatter(normal_encodings[:, 0], normal_encodings[:, 1], alpha=0.6)
axes[1].set_xlabel('Dimension 0')
axes[1].set_ylabel('Dimension 1')
axes[1].set_title('Distribution 2D des Encodings Normaux')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2 Scénarios d'Attaque


In [ ]:
def generate_attack_scenarios():
    """Générer différents types d'attaques"""
    n_attacks = 100
    
    scenarios = {
        'Légère': np.random.randn(n_attacks, encoding_dim) * 1.5 + 2,
        'Modérée': np.random.randn(n_attacks, encoding_dim) * 3 + 4,
        'Forte': np.random.randn(n_attacks, encoding_dim) * 5 + 8,
    }
    
    return scenarios

In [ ]:
scenarios = generate_attack_scenarios()

# Visualiser les scénarios
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, data) in enumerate(scenarios.items()):
    ax = axes[idx]
    ax.scatter(normal_encodings[:, 0], normal_encodings[:, 1], 
            alpha=0.4, label='Normal', s=50)
    ax.scatter(data[:, 0], data[:, 1], 
            alpha=0.6, label='Anomalie', s=50, marker='^')
    ax.set_xlabel('Dimension 0')
    ax.set_ylabel('Dimension 1')
    ax.set_title(f'Attaque {name}')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Tests de Résistance

### 4.1 Impact du Threshold (β)

In [ ]:
thresholds = [0.5, 1.0, 2.0, 3.0, 5.0]
k_neighbors = 3
gamma = 0.5

results = []

for threshold in thresholds:
    print(f"\n{'='*50}")
    print(f"Testing Threshold β = {threshold}")
    print(f"{'='*50}")
    
    for scenario_name, anomaly_stream in scenarios.items():
        # Créer mémoire
        memory = MemoryModule(
            memory_size=memory_size,
            encoding_dim=encoding_dim,
            k_neighbors=k_neighbors,
            gamma=gamma
        )
        memory.initialize(normal_encodings)
        
        # Simuler attaque
        result = memory.simulate_poisoning(anomaly_stream, threshold, verbose=True)
        
        results.append({
            'threshold': threshold,
            'scenario': scenario_name,
            'protection_rate': result['protection_rate'],
            'contamination_rate': result['contamination_rate']
        })


### 4.2 Visualisation des Résultats


In [ ]:
import pandas as pd

df = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Protection rate
for scenario in scenarios.keys():
    data = df[df['scenario'] == scenario]
    axes[0].plot(data['threshold'], data['protection_rate'], 
                marker='o', label=scenario, linewidth=2)

axes[0].set_xlabel('Threshold (β)', fontsize=12)
axes[0].set_ylabel('Taux de Protection (%)', fontsize=12)
axes[0].set_title('Protection vs Threshold', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].axhline(y=80, color='g', linestyle='--', alpha=0.5, label='Cible 80%')

# Contamination rate
for scenario in scenarios.keys():
    data = df[df['scenario'] == scenario]
    axes[1].plot(data['threshold'], data['contamination_rate'], 
                marker='s', label=scenario, linewidth=2)

axes[1].set_xlabel('Threshold (β)', fontsize=12)
axes[1].set_ylabel('Taux de Contamination (%)', fontsize=12)
axes[1].set_title('Contamination vs Threshold', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].axhline(y=20, color='r', linestyle='--', alpha=0.5, label='Limite 20%')

plt.tight_layout()
plt.show()

### 4.3 Tableau Récapitulatif


In [ ]:
print("\n" + "="*70)
print("TABLEAU RÉCAPITULATIF")
print("="*70)

pivot = df.pivot_table(
    values='protection_rate',
    index='scenario',
    columns='threshold',
    aggfunc='mean'
)

print("\nTaux de Protection (%):")
print(pivot.round(1))

print("\n" + "-"*70)

pivot_contam = df.pivot_table(
    values='contamination_rate',
    index='scenario',
    columns='threshold',
    aggfunc='mean'
)

print("\nTaux de Contamination (%):")
print(pivot_contam.round(1))

## 5. Impact de Gamma

### 5.1 Tests avec différentes valeurs de γ

In [ ]:
gamma_values = [0.0, 0.3, 0.5, 0.7, 0.9]
fixed_threshold = 2.0

gamma_results = []

print(f"\n{'='*50}")
print(f"Test Impact de Gamma (β fixe = {fixed_threshold})")
print(f"{'='*50}")

for gamma in gamma_values:
    memory = MemoryModule(
        memory_size=memory_size,
        encoding_dim=encoding_dim,
        k_neighbors=k_neighbors,
        gamma=gamma
    )
    memory.initialize(normal_encodings)
    
    # Utiliser attaque modérée
    result = memory.simulate_poisoning(
        scenarios['Modérée'], 
        fixed_threshold, 
        verbose=False
    )
    
    gamma_results.append({
        'gamma': gamma,
        'protection_rate': result['protection_rate'],
        'contamination_rate': result['contamination_rate']
    })
    
    print(f"γ = {gamma:.1f}: Protection = {result['protection_rate']:.1f}%")


### 5.2 Visualisation Impact Gamma


In [ ]:
df_gamma = pd.DataFrame(gamma_results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(df_gamma['gamma'], df_gamma['protection_rate'], 
        'o-', linewidth=2, markersize=10, color='green')
ax1.set_xlabel('Gamma (γ)', fontsize=12)
ax1.set_ylabel('Taux de Protection (%)', fontsize=12)
ax1.set_title('Impact de γ sur la Protection', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)
ax1.axhline(y=80, color='g', linestyle='--', alpha=0.5)

ax2.plot(df_gamma['gamma'], df_gamma['contamination_rate'], 
        's-', linewidth=2, markersize=10, color='red')
ax2.set_xlabel('Gamma (γ)', fontsize=12)
ax2.set_ylabel('Taux de Contamination (%)', fontsize=12)
ax2.set_title('Impact de γ sur la Contamination', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.axhline(y=20, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 6. Conclusions et Recommandations


In [ ]:
"""
### Résultats Clés:

1. **Impact du Threshold (β)**:
   - β = 0.5: Protection faible (~40-60%)
   - β = 2.0: Protection modérée (~70-85%)
   - β = 5.0: Protection forte (~90-95%)
   
2. **Impact de Gamma (γ)**:
   - γ = 0.0: Tous les voisins ont le même poids
   - γ > 0.5: Les voisins proches dominent le score
   - Meilleure protection avec γ élevé (0.7-0.9)

3. **Scénarios d'Attaque**:
   - Attaque Légère: Plus difficile à détecter
   - Attaque Forte: Facilement rejetée
   - Trade-off entre sensibilité et faux positifs

### Recommandations:

✅ **Pour une protection optimale**:
   - Utiliser β = 2.0 à 3.0
   - Utiliser γ = 0.5 à 0.7
   - Monitorer le taux de contamination

⚠️ **Limites**:
   - Un β trop élevé peut rejeter des points normaux
   - Un β trop bas permet la contamination
   - Ajuster selon le contexte applicatif
"""

print("\n" + "="*70)
print("✅ ANALYSE TERMINÉE")
print("="*70)